# Descenso por Gradiente en Regresión Lineal
Este notebook demuestra paso a paso cómo el descenso por gradiente ajusta una recta de regresión lineal minimizando el error cuadrático medio (MSE).

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Para reproducibilidad
np.random.seed(42)

## Generación de datos sintéticos

In [2]:
# Parámetros “reales” de la recta
true_w, true_b = 3.5, -8

# Generamos 100 observaciones
X = np.linspace(0, 10, 100)
noise = np.random.normal(0, 2, size=X.shape)
y = true_w * X + true_b + noise

## Funciones auxiliares

In [3]:
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def gradients(X, y_true, y_pred):
    n = len(X)
    dw = (-2/n) * np.sum(X * (y_true - y_pred))
    db = (-2/n) * np.sum(y_true - y_pred)
    return dw, db

def plot_line(ax, w, b, label=None, **kwargs):
    xs = np.array([X.min(), X.max()])
    ys = w * xs + b
    ax.plot(xs, ys, label=label, **kwargs)

## Descenso por Gradiente (loop iterativo)

In [ ]:
# Hiperparámetros
alpha = 0.01       # tasa de aprendizaje
n_iter = 200       # iteraciones

# Inicialización aleatoria
w, b = np.random.randn(), np.random.randn()

# Para visualizar la evolución
history_w, history_b, history_cost = [], [], []

for i in range(n_iter):
    y_pred = w * X + b
    cost = mse(y, y_pred)

    history_w.append(w)
    history_b.append(b)
    history_cost.append(cost)

    dw, db = gradients(X, y, y_pred)
    w -= alpha * dw
    b -= alpha * db

    if (i+1) % 25 == 0:
        print(f"Iter {i+1:3d}: MSE={cost:.3f}, w={w:.3f}, b={b:.3f}")

## Curva de aprendizaje

In [ ]:
plt.figure()
plt.plot(range(n_iter), history_cost)
plt.xlabel('Iteración')
plt.ylabel('MSE')
plt.title('Disminución del error durante el entrenamiento')
plt.grid(True)
plt.show()

## Recta final vs. datos

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X, y, s=20, alpha=0.6, label='datos')
plot_line(ax, w, b, label='ajuste final')
ax.legend()
ax.set_title('Regresión lineal ajustada')
plt.show()

## Animación de la evolución de la recta (opcional)

In [ ]:
from matplotlib.animation import FuncAnimation

fig, ax = plt.subplots()
ax.scatter(X, y, s=20, alpha=0.6)
line, = ax.plot([], [], lw=2)

def init():
    ax.set_xlim(X.min(), X.max())
    ax.set_ylim(y.min()-5, y.max()+5)
    return line,

def update(frame):
    w_i, b_i = history_w[frame], history_b[frame]
    xs = np.array([X.min(), X.max()])
    ys = w_i * xs + b_i
    line.set_data(xs, ys)
    return line,

ani = FuncAnimation(fig, update, frames=len(history_w),
                    init_func=init, blit=True, interval=50)

# Para mostrar en Jupyter simplemente ejecuta la celda; para guardar:
# ani.save('gradiente_lineal.gif', writer='pillow')
plt.close()  # evita doble visualización en algunos entornos
ani
ani.save('gradiente_lineal.gif', writer='pillow')

In [15]:
# =========================================================
# Animación doble: línea ajustada + punto que desciende la curva de pérdida
# =========================================================
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

# -----------------------------------------------------------------
# Curva parabólica del MSE en función de la pendiente (b fijo)
# -----------------------------------------------------------------
if len(history_w) == 0 or len(history_b) == 0:
    raise ValueError("history_w/history_b están vacíos. Ejecuta primero la celda de entrenamiento.")

b_fixed = history_b[-1]  # fijamos b para dibujar MSE(w)
w_grid  = np.linspace(min(history_w) - 1, max(history_w) + 1, 400)
loss_curve = [mse(y, w * X + b_fixed) for w in w_grid]

# -----------------------------------------------------------------
# Figura con dos subplots
# -----------------------------------------------------------------
fig, (ax_data, ax_loss) = plt.subplots(1, 2, figsize=(10, 4))

# 1) Datos y línea
ax_data.scatter(X, y, s=20, alpha=0.6)
line, = ax_data.plot([], [], lw=2, color='r')
ax_data.set_xlim(X.min(), X.max())
ax_data.set_ylim(y.min() - 5, y.max() + 5)
ax_data.set_title("Ajuste de la recta")

# 2) Curva de pérdida y punto móvil
ax_loss.plot(w_grid, loss_curve, lw=1.5)
point, = ax_loss.plot([], [], 'ro')
ax_loss.set_xlabel("Pendiente w")
ax_loss.set_ylabel("MSE")
ax_loss.set_title("Descenso por la curva de pérdida")

# -----------------------------------------------------------------
# Funciones de animación
# -----------------------------------------------------------------
def init():
    line.set_data([], [])
    point.set_data([], [])
    return line, point

def update(frame):
    w_i, b_i = history_w[frame], history_b[frame]

    # Línea sobre los datos
    xs = np.array([X.min(), X.max()])
    ys = w_i * xs + b_i
    line.set_data(xs, ys)

    # Punto sobre la curva de pérdida  ← AQUÍ
    current_loss = mse(y, w_i * X + b_fixed)
    point.set_data([w_i], [current_loss])      # <- usa listas o arrays

    return line, point


ani = FuncAnimation(fig, update, frames=len(history_w),
                    init_func=init, blit=True, interval=20)

plt.close()  # evita mostrar frame estático duplicado
ani.save('gradiente_lineal_doble.gif', writer='pillow')
